In [3]:
import pandas as pd
import chardet

In [7]:
with open("dataset_1382851.csv", "rb") as f:
    result = chardet.detect(f.read(100000))
    encoding = result["encoding"]

print(f"Detected encoding: {encoding}")

df = pd.read_csv("dataset_1382851.csv", encoding=encoding, delimiter=";")


Detected encoding: ISO-8859-1


In [9]:
df

,s_696_1,s_696_2,s_698,s_25,s_132,s_133,s_134,s_919,s_700,s_135,...,v_4,v_5,v_6,v_7,email,statoverall_1,statoverall_2,statoverall_3,statoverall_4,statoverall_5
0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,...,1,1,14~,"Høgevej 8, 6705 Esbjerg Ø",ses@business.aau.dk,0,0,0,1,0
1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,...,1,1,1-5~,"Østre Stationsvej 41P, 5000 Odense C",ses@business.aau.dk,0,0,0,1,0
2,0,1,NaN,NaN,NaN,NaN,NaN,NaN,1,Vognmænd og speditører med 6+ biler,...,1,1,2~,"Store Torv 17 1, 3700 Rønne",ses@business.aau.dk,0,0,0,1,0
3,0,1,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,...,1,1,1,"Kjellerupsgade 24, 9000 Aalborg",ses@business.aau.dk,0,0,0,1,0
4,0,1,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,...,1,1,1~,"Traps Allé 8, 2500 Valby",ses@business.aau.dk,0,0,0,1,0
5,0,1,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,...,2,1,NaN,"Hallindskovvej 60, 5700 Svendborg",ses@business.aau.dk,0,0,0,1,0
6,0,1,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,...,1,1,1-5~,"Brandevej 10 dør 11, 9220 Aalborg Øst",ses@business.aau.dk,0,0,0,1,0
7,1,1,1.0,"Sejlbådsejere der vægter sikkerhed, innovation...",NaN,NaN,NaN,NaN,1,Bådeværfter (Sejlbåde og yachtbyggere),...,1,1,NaN,"Kirkerupvej 39, 4000 Roskilde",ses@business.aau.dk,0,0,0,1,0
